 **Project 4: LLM Project Activity - Topic Modeling**

### **Week 22** Project 4: 1-Preprocessing
* Choose the project topic you will work on throughout this course and brainstorm for project ideas. (9.1)
* Configure the project workspace and access the dataset specific to the project topic. (9.2)

In [ ]:
from datasets import load_dataset

In [ ]:
!pip install -U fsspec datasets

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
ds = load_dataset('SetFit/20_newsgroups')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/734 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/8.91M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11314 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7532 [00:00<?, ? examples/s]

In [ ]:
#see available splits for data set
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 11314
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 7532
    })
})

In [ ]:
ds['train'][0]

{'text': 'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.',
 'label': 7,
 'label_text': 'rec.autos'}

In [ ]:
ds['train'].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_text': Value(dtype='string', id=None)}

In [ ]:
import pandas as pd
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [ ]:
# preprocessing HTML tag removal

import pandas as pd
from bs4 import BeautifulSoup

# Assuming 'ds' is a Hugging Face dataset dictionary with 'train' and 'test' splits
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

# Function to remove HTML tags using BeautifulSoup
def remove_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Apply the HTML cleaning to the 'text' column
ds_train['text'] = ds_train['text'].apply(remove_html)
ds_test['text'] = ds_test['text'].apply(remove_html)

# Optional: print first few rows to check
print(ds_train.head())
print(ds_test.head())

                                                text  label  \
0  I was wondering if anyone out there could enli...      7   
1  A fair number of brave souls who upgraded thei...      4   
2  well folks, my mac plus finally gave up the gh...      4   
3  \nDo you have Weitek's address/phone number?  ...      1   
4  From article , by tombaker@world.std.com (Tom ...     14   

              label_text  
0              rec.autos  
1  comp.sys.mac.hardware  
2  comp.sys.mac.hardware  
3          comp.graphics  
4              sci.space  
                                                text  label  \
0  I am a little confused on all of the models of...      7   
1  I'm not familiar at all with the format of the...      5   
2                                \nIn a word, yes.\n      0   
3  \nThey were attacking the Iraqis to drive them...     17   
4  \nI've just spent two solid months arguing tha...     19   

              label_text  
0              rec.autos  
1         comp.windows.x  

In [ ]:
# preprocessing removing punctuation
import string

# Function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Apply to a DataFrame column (e.g., 'text')
ds_train['text'] = ds_train['text'].apply(remove_punctuation)
ds_test['text'] = ds_test['text'].apply(remove_punctuation)

In [ ]:
import nltk

# Delete and re-download punkt
nltk.download('punkt', force=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# preprocessing tokenize, convert lowercase, remove numbers/punctuation, filter stop words (tokenization tehcnically part next notebook, leaving for completeness here but included next step as well)
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords_simple(text):
    tokens = text.lower().split()  # basic tokenization
    tokens = [word.strip(string.punctuation) for word in tokens if word.isalpha()]
    filtered = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered)

ds_train['text'] = ds_train['text'].apply(remove_stopwords_simple)
ds_test['text'] = ds_test['text'].apply(remove_stopwords_simple)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Convert Dataset to pandas DF for exploration, filtering, quick modifications
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("SetFit/20_newsgroups", split="train")

# Convert to DataFrame
df = dataset.to_pandas()

# Explore it
df.head()

Repo card metadata block was not found. Setting CardData to empty.


,text,label,label_text
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space


In [ ]:
#convert back to DF hugging Face Dataset
from datasets import Dataset

new_dataset = Dataset.from_pandas(df)

In [ ]:
from datasets import Dataset, DatasetDict
# assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)
# reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
        'train': train,
        'test': test
    }
)
# view the resulting dataset dict object
new_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 11314
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 7532
    })
})

In [24]:
from google.colab import files
uploaded = files.upload()

Saving Project_4_1_preprocessing.ipynb to Project_4_1_preprocessing.ipynb


In [27]:
import nbformat

notebook_path = '/content/Project_4_1_preprocessing.ipynb'  # Replace with actual name if different
cleaned_path = '/content/Project_4_1_preprocessing.ipynb_clean.ipynb'

with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

nb.metadata.pop('widgets', None)

with open(cleaned_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("Cleaned notebook saved to: C:\\Users\\alial\\OneDrive\\Desktop\\LHL\\Project 4")

Cleaned notebook saved to: C:\Users\alial\OneDrive\Desktop\LHL\Project 4
